In [ ]:
from IPython.display import clear_output
import numpy as np

Regressor class

In [599]:
class Regressor:

    def __init__(self, A_DIM, R_DIM):
        self.A_DIM = A_DIM
        self.R_DIM = R_DIM
        self.W = np.array([np.random.random(
            (self.A_DIM, self.R_DIM)), np.random.random((self.R_DIM, self.A_DIM))])

    def validate(self, a, b):
        self.variance, self.max_error, self.accuracy = self.metrics = self.compare_vector(
            a, b)

    def compare_vector(self, a, b):
        c = a - b
        c = np.squeeze(c)
        v = np.var(c)
        m = np.max(c)
        a = np.exp(- v / m) * 100
        return v, m, a

    def dropout_matrix(self, percent, shape):
        x = np.random.choice(2, shape, p=[1 - percent, percent])
        return x

    def fitUtil(self, training_data, training_label, alpha=0.0001, retain=1):

        hidden_layer = (training_data * self.dropout_matrix(retain,
                        training_data.shape[1])).dot(self.W[0])
        predicted_label = (
            hidden_layer * self.dropout_matrix(retain, hidden_layer.shape[1])).dot(self.W[1])

        error = training_label - predicted_label

        dW0 = - \
            training_data.T.dot(
                2 * error.dot(self.W[1].T)) / training_data.size
        dW1 = - hidden_layer.T.dot(2 * error) / training_data.size

        self.W[0] -= dW0 * alpha
        self.W[1] -= dW1 * alpha

    # Applies cross validation
    def fit(self, vecs, vecsy, epochs, **kwargs):

        vecs = np.array(np.split(vecs, 10))
        vecsy = np.array(np.split(vecsy, 10))

        for i in range(10):

            c_v_training_data, c_v_training_label = np.vstack(
                np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0))
            c_v_testing_data, c_v_testing_label = vecs[i], vecsy[i]

            for epoch in range(epochs // 10):
                self.fitUtil(c_v_training_data, c_v_training_label, **kwargs)

            clear_output()
            self.validate(self.predict(c_v_testing_data), c_v_testing_label)
            print("Metrics", i, ":", self.metrics)

    def predict(self, data):
        return data.dot(self.W[0]).dot(self.W[1])

    def __str__(self):
        return '\n'.join(["Error variance " + str(self.error_varience),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])

    def __repr__(self):
        return '\n'.join(["Error variance " + str(self.error_varience),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])


In [ ]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [805]:
class DataGenerator:
    def __init__(self, dim_a, dim_b):
        self.actual_dim = dim_a
        self.reduced_dim = dim_b


    def generate(self, data_amplitude = 1, data_count = 1000, rotation_count = 0, noise = 0):
        
        vecs = (np.random.random((data_count, self.reduced_dim)) * 2 * data_amplitude) - data_amplitude
        
        vecs = np.append(vecs, np.random.random((data_count, self.actual_dim - self.reduced_dim)) * noise * data_amplitude, axis = 1)
        
        for i in range(rotation_count):
            vecs = vecs.dot(self.get_rotational_matrix())
        return vecs

    def get_rotational_matrix(self, theta=None, axes=None):

        if theta is None:
            theta = 2 * np.pi * np.random.random()

        if axes is None:
            axes = np.random.choice(self.actual_dim, 2, replace=False)

        i, j = axes[0], axes[1]

        rotational_matrix = np.identity(self.actual_dim)
        rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
        rotational_matrix[i][j] = np.sin(theta)
        rotational_matrix[j][i] = -np.sin(theta)

        return rotational_matrix

In [804]:
regressor = Regressor(4, 3)
datagen = DataGenerator(4, 3)

C:\Users\joseg\AppData\Local\Temp/ipykernel_14656/2225721749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.W = np.array([np.random.random(


In [797]:

vecs = datagen.generate(1, 1000, 12, noise=1)

In [799]:
regressor.fit(vecs, vecs, 80000, retain=0.8, alpha = 0.001)
v1 = datagen.generate(1, 3000, 12344, noise=0.5)
regressor.compare_vector(regressor.predict(v1), v1)

Metrics 9 : (0.09487893248040292, 1.0363848334792447, 91.25175373770882)


(0.07288200195705359, 1.101568508775657, 93.59792010457724)

In [801]:
v1 = datagen.generate(1, 3000, 112, noise=0)
regressor.compare_vector(regressor.predict(v1), v1)

(0.07335065739029394, 0.7960314953317323, 91.1972519804229)

In [778]:
regressor.W[0].dot(regressor.W[1])

array([[ 1.1408,  0.0883, -0.019 ,  0.4926,  0.4058, -0.24  , -0.8264],
       [-0.0484,  1.1413,  0.0249, -0.0248, -0.1063,  0.5901, -0.061 ],
       [ 0.0137,  0.059 ,  1.2615,  0.0028,  0.038 , -0.1398,  0.0214],
       [ 0.0942,  0.0616,  0.0036,  1.0683, -0.11  , -0.1883,  0.3886],
       [ 0.1729,  0.0142,  0.0235, -0.3861,  1.0524,  0.0011,  0.6553],
       [ 0.1303,  0.3669, -0.0118,  0.0584,  0.2436,  0.1435,  0.117 ],
       [-0.1409,  0.018 , -0.0106,  0.3599,  0.2457, -0.052 ,  0.6662]])